# Example to compute image cross spectrum between different looks within a WV Sentinel-1 SLC product
in WV images (20 km X 20 km) there is only one burst (thus no overlapping area).


In [ ]:
import xsar
import warnings
import logging
from importlib import reload
reload(logging)
logging.basicConfig(level=logging.INFO)
logging.info('go')
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
#tmp = xsar.get_test_file('S1A_WV_SLC__1SSV_20160510T101603_20160510T102143_011195_010EA1_Z010.SAFE')
tmp = xsar.get_test_file('S1A_WV_SLC__1SSV_20191005T100804_20191005T101314_029322_035533_0597.SAFE') # very small product
# print(tmp)
# tmp = '/home/datawork-cersat-public/cache/project/mpc-sentinel1/data/esa/sentinel-1a/L2/WV/S1A_WV_OCN__2S/2016/131/S1A_WV_OCN__2SSV_20160510T101603_20160510T102143_011195_010EA1_F972.SAFE'
# tmp = '/home/datawork-cersat-public/cache/project/mpc-sentinel1/data/esa/sentinel-1a/L1/WV/S1A_WV_SLC__1S/2016/131/S1A_WV_SLC__1SSV_20160510T101603_20160510T102143_011195_010EA1_7038.SAFE'
# tmp = '/home/datawork-cersat-public/cache/project/mpc-sentinel1/data/esa/sentinel-1a/L1/WV/S1A_WV_SLC__1S/2023/014/S1A_WV_SLC__1SSV_20230114T130811_20230114T133351_046780_059BC5_FED5.SAFE'

strwv = 'SENTINEL1_DS:'+tmp+':WV_017'
print(strwv)
xsar_instance = xsar.Sentinel1Dataset(strwv)
xsar_instance


In [ ]:
xsar_instance.dataset

In [ ]:
xsar_instance.dataset.digital_number.values[0:10,0:10]

# display digital numbers

In [ ]:
import holoviews as hv
import numpy as np
from holoviews.operation.datashader import datashade,rasterize
hv.extension('bokeh')

In [ ]:
tmp = abs(xsar_instance.dataset['digital_number'].sel(pol='VV'))
DN = tmp.rolling({"sample":10,"line":10}).mean()
DN

In [ ]:
rasterize(hv.Image(DN.values).opts(width=800,height=700,cmap='Greys',colorbar=True))# clim=(0,40)

# compute the periodograms and the looks inside the periodograms to get a cross spectra

In [ ]:
from xsarslc.processing.xspectra import compute_WV_intraburst_xspectra
import time
t0 = time.time()
xs0 = compute_WV_intraburst_xspectra(dt=xsar_instance.datatree,
                                          polarization='VV',periodo_width={"line":2000,"sample":2000},
                                          periodo_overlap={"line":1000,"sample":1000})
print('time to compute x-spectra on WV :%1.1f sec'%(time.time()-t0))
xs0

In [ ]:
import numpy as np
from xsarslc.processing import xspectra
xs = xs0.swap_dims({'freq_line': 'k_az', 'freq_sample': 'k_rg'})
xs = xspectra.symmetrize_xspectrum(xs, dim_range='k_rg', dim_azimuth='k_az')

############################################ real part ############################
coS=np.abs(xs['xspectra_2tau'].mean(dim='2tau').real) 
coS_reduced = coS.where(np.logical_and(np.abs(coS.k_rg)<=0.14, np.abs(coS.k_az)<=0.14), drop=True)
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib import colors as mcolors
cmap = mcolors.LinearSegmentedColormap.from_list("", ["white","violet","mediumpurple","cyan","springgreen","yellow","red"])
PuOr = plt.get_cmap('PuOr')
plt.figure(figsize=(8,6))
#coS_reduced_av=coS_reduced.rolling(k_rg=3, center=True).mean().rolling(k_az=3, center=True).mean()
coS_reduced.plot(cmap=cmap, levels=20, vmin=0)

plt.grid()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.axis('scaled')

############################################ imag.part ############################
ims = xs['xspectra_2tau'].mean(dim='2tau').imag.squeeze()
xS_red = ims.where(np.logical_and(np.abs(ims.k_rg)<=0.14, np.abs(ims.k_az)<=0.14), drop=True)
#xS_av=xS_red.rolling(k_rg=3, center=True).mean().rolling(k_az=3, center=True).mean()

plt.figure(figsize=(8,6))
xS_red.plot(x='k_rg',cmap=PuOr)
plt.grid()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.axis('scaled')